Import numpy for numerical computations

In [2]:
import numpy as np

First idea: At each step, take outgoing link at random. This can be viewed as a Markov chain on a finite state space.
Suppose we have a (very) small internet with 4 webpages that contain the following hyperlinks:

- Page 1 has links to Pages 2 and 3
- Page 2 has links to Pages 1, 2, and 3
- Page 3 has a link to Page 1
- Page 4 has a link to Page 2

The transition matrix for the corresponding Markov chain is:
$$P = \begin{bmatrix} 0 & 1/3 & 1 & 0 \\ 1/2 & 0 & 0 & 0 \\ 1/2 & 1/3 & 0 & 1 \\ 0 & 1/3 & 0 & 0 \end{bmatrix}$$

In [7]:
P = [[0,1/3,1,0],[1/2,0,0,0],[1/2,1/3,0,1],[0,1/3,0,0]]

## Compute eigenvalues and eigenvectors of P using numpy
[eigvals,eigvecs] = np.linalg.eig(P)

## Note that eigenvalues may in general be complex, but their magnitude should be between 0 and 1
print(np.abs(eigvals))

[1.         0.8394315  0.44558621 0.44558621]


Unfortunately, there may be multiple steady state vectors, as in this case where

- Page 1 has a link to Page 2
- Page 2 has a link to Page 1
- Page 3 has a link to Page 4
- Page 4 has a link to Page 3

which leads to the transition matrix
$$P = \begin{bmatrix} 0 & 1 & 0 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$$

In [8]:
P = [[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,0,1,0]]

## Compute eigenvalues and eigenvectors of P
[V,D] = np.linalg.eig(P)
print(V[0])
print(D[:,0])
print(V[2])
print(D[:,2])

1.0
[0.70710678 0.70710678 0.         0.        ]
1.0
[0.         0.         0.70710678 0.70710678]


To avoid the problem of multiple steady state vectors, we can instead construct a transition matrix that is a regular stochastic matrix, which has only positive entries. One way of doing this is by adding a probability $$\alpha$$ of following one of the outgoing links, with a probability $$1-\alpha$$ of teleporting to another random webpage regardless of whether or not there is a link.

Running this on a radomly generated $50\times50$ example below:

In [15]:
from numpy.random import Generator, PCG64
rng = Generator(PCG64())

alpha = .85
N = 50
P = rng.integers(2,size=[N,N]) ## Generate an nxn random matrix of 0s and 1s
P = P/np.sum(P,0)              ## Normalize so that each column is zero
teleport = np.ones([N,N]) / N
[V,D] = np.linalg.eig(alpha*P + (1-alpha)*teleport)
print(np.abs(V))

[1.         0.1138354  0.1138354  0.1153755  0.1153755  0.11473845
 0.11473845 0.1065874  0.1065874  0.10223132 0.10223132 0.124162
 0.08952477 0.08952477 0.10801213 0.10801213 0.10838933 0.10838933
 0.10651377 0.10651377 0.09320633 0.09320633 0.09079354 0.09079354
 0.0898665  0.0898665  0.08076035 0.08076035 0.0876453  0.0876453
 0.08358854 0.08358854 0.08366782 0.0796152  0.0796152  0.06372522
 0.06372522 0.0537339  0.0537339  0.0488352  0.06052507 0.06052507
 0.02021442 0.02021442 0.03283807 0.03283807 0.04545316 0.04545316
 0.02675437 0.02675437]


Above we should see only one eigenvalue with magnitude 1, and all other eigenvalues have magnitude between 0 and 1

How to compute ranking vector? Can do so using iterative process

In [17]:
def PageRank(P,alpha):
    # Rank pages in the web, given by matrix P, with a teleporting parameter of alpha. 
    N = np.size(P,1)
    x = np.ones(N)/N
    teleport = np.ones(N)/N
    maxIter = 100000
    for k in range(maxIter):
        x = alpha*P.dot(x) + (1-alpha)*teleport
    
    return x

In [18]:
alpha = .85
N = 100
P = rng.integers(2,size=[N,N])
P = P/np.sum(P,0)
x = PageRank(P,alpha)
print(x)

[0.01060043 0.01067501 0.00922863 0.01053159 0.00927021 0.00927028
 0.01094494 0.00867916 0.01060522 0.00777192 0.01040179 0.01042609
 0.01087005 0.00978826 0.01011405 0.01004143 0.00972503 0.01021163
 0.0096648  0.01078949 0.00940126 0.00992869 0.00800818 0.00928596
 0.0094977  0.00927572 0.01151225 0.00919604 0.01139111 0.01079805
 0.00895124 0.00920629 0.00960527 0.00999075 0.0105156  0.00988651
 0.00837149 0.01209206 0.00870791 0.00988515 0.00992839 0.01010159
 0.01035176 0.01136676 0.00922675 0.00957575 0.00995562 0.01053603
 0.01134778 0.01005141 0.00908504 0.00873303 0.0094535  0.01031518
 0.01080921 0.00862697 0.01092796 0.01054675 0.00969796 0.00912532
 0.00950293 0.0110433  0.01025793 0.01050908 0.00991139 0.01035763
 0.01206668 0.00974198 0.00831198 0.01027195 0.01058395 0.01093082
 0.00794084 0.01022685 0.01033138 0.00962162 0.00903742 0.01167363
 0.00928793 0.01108674 0.00996756 0.00893479 0.0104298  0.00909806
 0.01115378 0.01035933 0.00890277 0.00972988 0.01029731 0.0086